In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import json
import httpx
import string
import secrets

from time import sleep

from getpass import getpass

from deploy.tasks import DeploymentContext
from deploy.auth import token_to_payload

<IPython.core.display.Javascript object>

In [3]:
base_url = "https://deploy.staging.wersdoerfer.de"
username = "jochen"
password = getpass()
service = "podlove"

# get access_token
client = httpx.Client(base_url=base_url)
r = client.post("/token", data={"username": username, "password": password})
access_token = r.json()["access_token"]
access_token_headers = {"authorization": f"Bearer {access_token}"}

# get service token
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers={"authorization": f"Bearer {service_token}"}
client.headers = headers

 ····················


<IPython.core.display.Javascript object>

# Deploy

In [4]:
def show_details(details_url):
    r = client.get(details_url, headers=headers)
    rdata = r.json()
    len_steps = len([s for s in rdata["steps"] if s["state"] == "success"])
    started = rdata["started"]
    finished = rdata["finished"]
    print(f"{len_steps} {started} {finished}")

<IPython.core.display.Javascript object>

## Create Service

In [5]:
def create_secret(length=32):
    alphabet = string.ascii_letters + string.digits
    return ''.join(secrets.choice(alphabet) for i in range(length))

<IPython.core.display.Javascript object>

In [6]:
fqdn = "podlove.wp.staging.django-cast.com"
underscored_fqdn = fqdn.replace(".", "_")
site_id = f"wp_{underscored_fqdn}"
env = {
    "fqdn": fqdn,
    "site_id": site_id,
    "user_name": "wp_1",
    "database_name": site_id,
    "database_user": site_id,
    "database_password": create_secret(),
    "port": 12001,
    "auth_key": create_secret(),
    "secure_auth_key": create_secret(),
    "logged_in_key": create_secret(),
    "nonce_key": create_secret(),
    "auth_salt": create_secret(),
    "secure_auth_salt": create_secret(),
    "logged_in_salt": create_secret(),
    "nonce_salt": create_secret(),
}
env

{'fqdn': 'podlove.wp.staging.django-cast.com',
 'site_id': 'wp_podlove_wp_staging_django-cast_com',
 'user_name': 'wp_1',
 'database_name': 'wp_podlove_wp_staging_django-cast_com',
 'database_user': 'wp_podlove_wp_staging_django-cast_com',
 'database_password': 'ta84RE7N99EZb3GxTUYezkylUvdeum4A',
 'port': 12001,
 'auth_key': 'WWH4s7qDh0ccX1ACkyyGri5MWJPrCyce',
 'secure_auth_key': 'HfHaZjBrpSb1Ef4Caq1bMsVHy73KMNpU',
 'logged_in_key': 'vwctGNHyLnQO2poF6INxNNTjNiRQGYLG',
 'nonce_key': 'iFJXyutbdJEokEsBrEDNVIZOQFoiOXpV',
 'auth_salt': 'G3KJzpQVfez5yiQIhwEMDsgUkJ3QKFAX',
 'secure_auth_salt': 'Zarwsg5Kq115kYRALSFvwxUPBSjrMcnt',
 'logged_in_salt': '6AJgZRJMvEhAFhmfEtrupkcyCppdX3Cg',
 'nonce_salt': 'AtRxBfPS8HWmQ3YX6ja9WqTuIBsxNoBp'}

<IPython.core.display.Javascript object>

### Get Service Token

In [10]:
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers = {"authorization": f"Bearer {service_token}"}

<IPython.core.display.Javascript object>

In [12]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

# for i in range(3):
#     show_details(details_url)
#     sleep(1)

{'id': 194, 'service_id': 60, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-06-25T17:16:19.950488+00:00', 'finished': None, 'context': {'env': {'fqdn': 'podlove.wp.staging.django-cast.com', 'site_id': 'wp_podlove_wp_staging_django-cast_com', 'user_name': 'wp_1', 'database_name': 'wp_podlove_wp_staging_django-cast_com', 'database_user': 'wp_podlove_wp_staging_django-cast_com', 'database_password': 'ta84RE7N99EZb3GxTUYezkylUvdeum4A', 'port': 12001, 'auth_key': 'WWH4s7qDh0ccX1ACkyyGri5MWJPrCyce', 'secure_auth_key': 'HfHaZjBrpSb1Ef4Caq1bMsVHy73KMNpU', 'logged_in_key': 'vwctGNHyLnQO2poF6INxNNTjNiRQGYLG', 'nonce_key': 'iFJXyutbdJEokEsBrEDNVIZOQFoiOXpV', 'auth_salt': 'G3KJzpQVfez5yiQIhwEMDsgUkJ3QKFAX', 'secure_auth_salt': 'Zarwsg5Kq115kYRALSFvwxUPBSjrMcnt', 'logged_in_salt': '6AJgZRJMvEhAFhmfEtrupkcyCppdX3Cg', 'nonce_salt': 'AtRxBfPS8HWmQ3YX6ja9WqTuIBsxNoBp'}}, 'details': 'https://deploy.staging.wersdoerfer.de/deployments/194'}


<IPython.core.display.Javascript object>

## Remove Service

### Get Service Token

In [9]:
data = {"service": "podlove_remove", "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers={"authorization": f"Bearer {service_token}"}

<IPython.core.display.Javascript object>

In [11]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

{'id': 190, 'service_id': 61, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-06-25T16:46:16.315117+00:00', 'finished': None, 'context': {'env': {'fqdn': 'podlove.wp.staging.django-cast.com', 'site_id': 'wp_podlove_wp_staging_django-cast_com', 'user_name': 'wp_1', 'database_name': 'wp_podlove_wp_staging_django-cast_com', 'database_user': 'wp_podlove_wp_staging_django-cast_com', 'database_password': '4ilw3uiysnT21evwDnVF9YfpnKgIUFbu', 'port': 12001, 'auth_key': 'fcamRo2atmKkvmZGZMDU02ja1xcWcHCG', 'secure_auth_key': 'u2zSu4Nu60NIvVic4TAaE7ljKuknKY16', 'logged_in_key': '8X2mos56HGx6qNOw2xN0mYe7sHA3lBS3', 'nonce_key': 'xPwvdUkkO2IPFtsjrKHcQI42huDeQ6ry', 'auth_salt': '1csT6rVrVzFpUfrnSTUrQB6l0HEjWFnP', 'secure_auth_salt': 'dxApGBXS1gK6ZPZzBdSX4tVVfb2mY8uG', 'logged_in_salt': 'gABSOoSGXXOviMCYDhMp6j8lmdeFsMqC', 'nonce_salt': 'hLD6iFZC3uAqbDuk46cElyzbitXNVXX3'}}, 'details': 'https://deploy.staging.wersdoerfer.de/deployments/190'}


<IPython.core.display.Javascript object>

In [35]:
context.env

{'fqdn': 'foobar.staging.wersdoerfer.de',
 'database_name': 'cast_foobar.staging.wersdoerfer.de',
 'database_user': 'cast_foobar.staging.wersdoerfer.de',
 'database_password': '6It0n1QGoMM4GtAkGmRr',
 'secret_key': '2oCTxJIo9YBpWquiVG5Ks3cJHcIPKZUB',
 'port': 10001,
 'settings_file_name': 'foobar_staging_wersdoerfer_de'}

<IPython.core.display.Javascript object>

# Stuff

In [31]:
foo = {"bar": "baz"}

<IPython.core.display.Javascript object>

In [32]:
dict(foo, blub="bla")

{'bar': 'baz', 'blub': 'bla'}

<IPython.core.display.Javascript object>

In [33]:
dict(foo, bar="blub")

{'bar': 'blub'}

<IPython.core.display.Javascript object>